In [ ]:
img_extensions = ['.jpg', '.png', '.jpeg', '.webp', '.gif', '.bmp', '.tiff', '.ico', '.svg']
zip_extensions = ['.zip', '.rar', '.7z', '.tar', '.gz', '.bz2', '.xz']
manga_extensions = ['.cbz', '.cbr', '.cb7', '.cbt', '.cbg', '.cbb', '.cbf']

src_ext = img_extensions + zip_extensions + manga_extensions
print(src_ext)

In [ ]:
'''
메인 디렉토리가 /home/manga

/home/manga/(장르 별 하위 디렉토리)/(manga 디렉토리)/(이미지 파일들(압축파일도 있을 수 있음))
예시 
/home/manga/manga/manga디렉토리/이미지 파일들(압축파일도 있을 수 있음)
/home/manga/__manga2/manga디렉토리/이미지 파일들(압축파일도 있을 수 있음)
/home/manga/__그림그리기/manga디렉토리/이미지 파일들(압축파일도 있을 수 있음)
등등 여러개 있음

...

manga 확장자는 
img_extensions = ['.jpg', '.png', '.jpeg', '.webp', '.gif', '.bmp', '.tiff', '.ico', '.svg']

manga 압축파일 
zip_extensions = ['.zip', '.rar', '.7z', '.tar', '.gz', '.bz2', '.xz']

1. 압축파일있는 디렉토리와 image 파일있는 디렉토리는 제외하고 이미지 파일이 없거나 이미지 파일 수가 10개 미만인 디렉토리를 찾아서 하위 디렉토리와 파일들은 /home/manga/__del 디렉토리로 이동하고 삭제
2. 압축파일이있는 디렉토리중 그림파일이 2개 이하면 삭제하고 압축파일에서 한장씩만 문자 안깨지게 UTF-8 인코딩으로 같은 디렉토리에 복사
3. manga 디렉토리 아래 이미지 파일들 숫자.jpg 숫자.webp 형식으로 이름 변경
4. sqlite3 데이터베이스 만들고 디렉토리 정보 저장
    'folder' : manga 디렉토리의 /home/manga/ 부분 제외한 디렉토리 경로
    'images' : manga 디렉토리 아래 이미지 파일 목록
    'zip_files' : manga 디렉토리 아래 압축파일 목록
    'file_date' : 이미지 파일들 생성일자 중 가장 오래된 날짜
    'create_date' : db 생성일자
    'update_date' : db 수정일자
    'tag' : ''
    
    
    
'''


In [1]:
y = 'xxx.jpg'
y[y.rfind('.')+1:]

'jpg'

In [6]:
import os
import shutil
import sqlite3
from datetime import datetime
import pytz
import zipfile
import py7zr
import rarfile
import patoolib
from pathlib import Path

# 한국 시간대 설정
KST = pytz.timezone('Asia/Seoul')

# 파일 확장자 정의
img_extensions = ['.jpg', '.png', '.jpeg', '.webp', '.gif', '.bmp', '.tiff', '.ico', '.svg']
zip_extensions = ['.zip', '.rar', '.7z', '.tar', '.gz', '.bz2', '.xz', '.alz']

def get_file_list(directory):
    """디렉토리 내의 이미지와 압축 파일 목록을 반환"""
    images = []
    zip_files = []
    
    for file in os.listdir(directory):
        file_path = os.path.join(directory, file)
        if os.path.isfile(file_path):
            ext = os.path.splitext(file)[1].lower()
            if ext in img_extensions:
                images.append(file_path)
            elif ext in zip_extensions:
                zip_files.append(file_path)
                
    return images, zip_files

def process_manga_directory(manga_path):
    """개별 만화 디렉토리 처리"""
    images, zip_files = get_file_list(manga_path)
    
    # 조건 1: 이미지가 없거나 10개 미만이고 압축파일이 없는 경우
    if len(zip_files) == 0 and (len(images) == 0 or len(images) < 10):
        del_path = os.path.join('/home/manga/__del', os.path.relpath(manga_path, '/home/manga'))
        os.makedirs(os.path.dirname(del_path), exist_ok=True)
        try:
            shutil.move(manga_path, del_path)
        except Exception as e:
            print(f"Error moving {manga_path} to {del_path}: {str(e)}")
        return
    
    # 조건 2: 압축파일이 있고 이미지가 2개 이하인 경우
    if zip_files and len(images) <= 2:
        for img in images:
            os.remove(img)
        
        for zip_file in zip_files:
            try:
                extract_first_image(zip_file, manga_path)
            except Exception as e:
                print(f"Error processing {zip_file}: {str(e)}")
    
    # 조건 3: 이미지 파일 이름 변경
    rename_images(manga_path)

def extract_first_image(zip_file, dest_dir):
    """압축 파일에서 첫 번째 이미지 추출"""
    ext = os.path.splitext(zip_file)[1].lower()
    
    try:
        if ext == '.zip':
            with zipfile.ZipFile(zip_file, 'r') as zf:
                for file in zf.namelist():
                    if any(file.lower().endswith(ext) for ext in img_extensions):
                        zf.extract(file, dest_dir)
                        break
        elif ext == '.7z':
            with py7zr.SevenZipFile(zip_file, 'r') as sz:
                for file in sz.getnames():
                    if any(file.lower().endswith(ext) for ext in img_extensions):
                        sz.extract(dest_dir, [file])
                        break
        elif ext == '.rar':
            with rarfile.RarFile(zip_file, 'r') as rf:
                for file in rf.namelist():
                    if any(file.lower().endswith(ext) for ext in img_extensions):
                        rf.extract(file, dest_dir)
                        break
        else:
            patoolib.extract_archive(zip_file, outdir=dest_dir, interactive=False)
    except Exception as e:
        print(f"Error extracting {zip_file}: {str(e)}")

def rename_images(directory):
    """이미지 파일 이름을 숫자 형식으로 변경"""
    images = [f for f in os.listdir(directory) if any(f.lower().endswith(ext) for ext in img_extensions)]
    images.sort()
    
    for i, img in enumerate(images, 1):
        ext = os.path.splitext(img)[1].lower()
        if ext == '.webp':
            new_name = f"{i}.webp"
        else:
            new_name = f"{i}.jpg"
        old_path = os.path.join(directory, img)
        new_path = os.path.join(directory, new_name)
        os.rename(old_path, new_path)

def update_database(manga_path):
    """SQLite 데이터베이스 업데이트"""
    conn = sqlite3.connect('manga.db')
    cursor = conn.cursor()
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS manga_dirs (
        folder TEXT PRIMARY KEY,
        images TEXT,
        zip_files TEXT,
        file_date TEXT,
        create_date TEXT,
        update_date TEXT,
        tag TEXT
    )
    ''')
    
    images, zip_files = get_file_list(manga_path)
    relative_path = os.path.relpath(manga_path, '/home/manga')
    
    if images:
        file_date = min(datetime.fromtimestamp(os.path.getmtime(img), KST) 
                      for img in images).strftime('%Y-%m-%d %H:%M:%S')
    else:
        file_date = ''
        
    now = datetime.now(KST).strftime('%Y-%m-%d %H:%M:%S')
    
    cursor.execute('''
    INSERT OR REPLACE INTO manga_dirs 
    (folder, images, zip_files, file_date, create_date, update_date, tag)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (
        relative_path,
        ','.join([os.path.basename(img) for img in images]),
        ','.join([os.path.basename(zip_f) for zip_f in zip_files]),
        file_date,
        now,
        now,
        ''
    ))
    
    conn.commit()
    conn.close()

def main():
    manga_root = '/home/manga'
    
    # 모든 장르 디렉토리 순회
    for genre_dir in os.listdir(manga_root):
        genre_path = os.path.join(manga_root, genre_dir)
        if not os.path.isdir(genre_path):
            continue
            
        # 각 장르 디렉토리 내의 만화 디렉토리 순회
        for manga_dir in os.listdir(genre_path):
            manga_path = os.path.join(genre_path, manga_dir)
            if not os.path.isdir(manga_path):
                continue
                
            # 만화 디렉토리 처리
            process_manga_directory(manga_path)
            
            # 데이터베이스 업데이트
            if os.path.exists(manga_path):  # 삭제되지 않은 경우만
                update_database(manga_path)

if __name__ == "__main__":
    main()

Error moving /home/manga/__del/__del to /home/manga/__del/__del/__del: Cannot move a directory '/home/manga/__del/__del' into itself '/home/manga/__del/__del/__del'.
Error extracting /home/manga/__manga2/神坂公平/번역_神坂公平_sisters～여름의_마지막_날～.rar: Cannot find working tool
Error extracting /home/manga/__manga2/第6基地/번역_C80_第6基地_나코_라이프_꽃이_피는_첫걸음_동인지.rar: Cannot find working tool
Error extracting /home/manga/__manga2/菊一もんじ/번역_菊一もんじ_アナリスト.rar: Cannot find working tool
Error extracting /home/manga/__manga2/藤原俊一/팀H번역_藤原俊一_에프터_스쿨_상업지.rar: Cannot find working tool
Error extracting /home/manga/__manga2/軒下の猫屋/번역_C83_軒下の猫屋_アルデヒド_어둠의_불꽃에_안기고_싶어_중2병이라도_사랑이_하고_싶어_팀아키바.rar: Cannot find working tool


### video 폴더
/home/video/


In [4]:
pip install pytz py7zr rarfile patool

nginx service is running...
code service is running...
Note: you may need to restart the kernel to use updated packages.
